# State Management with StateMachine
This notebook demonstrates how to work with state management using a StateMachine implementation. We'll explore how to create, manage, and control workflow states in a structured way.

## What we'll learn:
- Basic state machine concepts and implementation
- Creating and connecting workflow steps
- Managing state transitions and data flow
- Working with routing and loops in state machines
- Understanding state snapshots and execution flow

### Setup

In [1]:
from typing import TypedDict
from lib.state_machine import (
    StateMachine,
    Step,
    EntryPoint,
    Termination,
)
import os 

## Basic State Machine Concepts
Let's start with a simple example that demonstrates the core concepts of our state machine:
1. Defining state schema
2. Creating steps
3. Connecting steps
4. Running the workflow

**Creating the Schema and the State Machine**

In [2]:
class Schema(TypedDict):
    """Schema defining the structure of our state.
    
    Attributes:
        input: The input value to process
        output: The processed output value
    """
    input: int
    output: int

In [3]:
# Create our state machine instance
workflow = StateMachine(Schema)

**Defining the logic for Steps**

In [4]:
def step_input(state: Schema) -> Schema:
    """First step: Increment the input value.
    
    Args:
        state: Current state containing input value
        
    Returns:
        Updated state with incremented value in output
    """
    return {"output": state["input"] + 1, "random": 10}



In [5]:
def step_double(state: Schema) -> Schema:
    """Second step: Double the previous output.
    
    Args:
        state: Current state containing output from previous step
        
    Returns:
        Updated state with doubled output value
    """
    return {"output": state["output"] * 2}


**Creating and Connecting Steps**

In [8]:
entry = EntryPoint()
s1 = Step("input", step_input)
s2 = Step("double", step_double)
termination = Termination()

In [7]:
workflow.add_steps([entry, s1, s2, termination])

In [9]:
workflow.connect(entry, s1)
workflow.connect(s1, s2)
workflow.connect(s2, termination)

In [10]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['input'])],
 'input': [Transition('input' -> ['double'])],
 'double': [Transition('double' -> ['__termination__'])]}

**Running the Workflow**

In [11]:
initial_state = {"input": 4}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: input
[StateMachine] Executing step: double
[StateMachine] Terminating: __termination__


Run('2c91f66f-57e8-4887-84ef-3664161495e7')

In [12]:
run_object.snapshots

[Snapshot('c527f7cc-6c7c-4493-acad-948e745d8e13') @ [2025-12-22 21:49:00.892199]: __entry__.State({'input': 4}),
 Snapshot('aefc86f9-77fd-4740-9197-702f519c3ada') @ [2025-12-22 21:49:00.892283]: input.State({'input': 4, 'output': 5}),
 Snapshot('8c6bd8e7-cd7a-4b68-9562-936aded23e30') @ [2025-12-22 21:49:00.892371]: double.State({'input': 4, 'output': 10})]

## Advanced State Management: Routing and Loops
Now we'll explore more complex state management patterns including:
- Conditional routing between steps
- Creating loops in the workflow
- Managing state through multiple iterations

In [13]:
class CounterSchema(TypedDict):
    """Schema for a counter-based workflow.
    
    Attributes:
        count: Current counter value
        max_value: Maximum value before termination
    """
    count: int
    max_value: int

In [14]:
workflow = StateMachine(CounterSchema)

In [15]:
def increment_counter(state: CounterSchema) -> CounterSchema:
    """Increment the counter value.
    
    Args:
        state: Current state with counter value
        
    Returns:
        Updated state with incremented counter
    """
    return {"count": state["count"] + 1}

In [16]:
# Create steps
entry = EntryPoint()
increment = Step("increment", increment_counter)
termination = Termination()

In [17]:
workflow.add_steps([entry, increment, termination])

In [18]:
# Router logic
def check_counter(state: CounterSchema) -> Step:
    """Determine next step based on counter value.
    
    Args:
        state: Current state with counter and max value
        
    Returns:
        Next step to execute (increment or terminate)
    """
    if state["count"] >= state["max_value"]:
        return termination
    return increment

In [19]:
# Connect steps with a loop in increment
workflow.connect(entry, increment)
workflow.connect(increment, [increment, termination], check_counter)

In [20]:
workflow.transitions

{'__entry__': [Transition('__entry__' -> ['increment'])],
 'increment': [Transition('increment' -> ['increment', '__termination__'])]}

In [21]:
initial_state = {"count": 0, "max_value": 3}
run_object = workflow.run(initial_state)
run_object

[StateMachine] Starting: __entry__
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Terminating: __termination__


Run('ae80be2c-96e1-4de4-bd6f-18ec7347feec')

In [22]:
run_object.snapshots

[Snapshot('a3ac0fcf-79f7-435a-8829-684ed9f3d2e1') @ [2025-12-22 21:51:18.194846]: __entry__.State({'count': 0, 'max_value': 3}),
 Snapshot('aadbaba4-c2f3-4e93-882d-ead88e9af0a5') @ [2025-12-22 21:51:18.194915]: increment.State({'count': 1, 'max_value': 3}),
 Snapshot('9d3135b4-c39c-44e4-8843-4a912d729d34') @ [2025-12-22 21:51:18.194975]: increment.State({'count': 2, 'max_value': 3}),
 Snapshot('2f6e379e-45a8-42d8-a028-0e3314f47813') @ [2025-12-22 21:51:18.195029]: increment.State({'count': 3, 'max_value': 3})]

In [23]:
run_object.get_final_state()

{'count': 3, 'max_value': 3}

In [27]:
initial_state = {"count": 3, "max_value": 12}
run_object = workflow.run(initial_state)
print(run_object.snapshots)
run_object.get_final_state()

[StateMachine] Starting: __entry__
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Executing step: increment
[StateMachine] Terminating: __termination__
[Snapshot('e783ebf6-fe00-488e-84fc-4eae99b28888') @ [2025-12-22 21:56:17.308420]: __entry__.State({'count': 3, 'max_value': 12}), Snapshot('6de86b73-c703-420e-9ae7-11343eaf044d') @ [2025-12-22 21:56:17.308521]: increment.State({'count': 4, 'max_value': 12}), Snapshot('7fc59a38-d6a2-4057-8c07-b19df08bac9e') @ [2025-12-22 21:56:17.308586]: increment.State({'count': 5, 'max_value': 12}), Snapshot('b7f8ecd0-52b8-4b16-81c1-07e194c83703') @ [2025-12-22 21:56:17.308640]: increment.State({'count': 6, 'max_value': 12}), Snapshot('36fba9fe-25c0-4325-be38-b

{'count': 12, 'max_value': 12}